# Name cloze test

In [3]:
from glob import glob
import pandas as pd
from nltk.tokenize import sent_tokenize
import textwrap
import random

import time
import backoff
import math
import openai

import seaborn as sns

import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.chunk import ne_chunk

def print_wrapped_text(text, width=70):
    texts = text.split('\n')
    if len(texts) > 1:
        for t in texts:
            print_wrapped_text(t, width=70)

    else:
        text = texts[0]
        wrapper = textwrap.TextWrapper(width=width)
        wrapped_text = wrapper.fill(text)
        print(wrapped_text)
    
with open('credentials.txt', encoding = 'utf-8') as f:
    organization = f.readline().strip()
    api_key = f.readline().strip()
    
client = openai.OpenAI(organization=organization, api_key=api_key)

In [2]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')


[nltk_data] Downloading package punkt to /Users/tunder/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/tunder/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /Users/tunder/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /Users/tunder/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

### Create questions and answers

First we need to have a function that loads a book and generates questions.

In [11]:
def extract_personal_names(sentence):
    # Tokenize and POS tag the words in the sentence
    words = nltk.word_tokenize(sentence)
    pos_tags = nltk.pos_tag(words)

    # Chunk words using named entity chunker
    tree = ne_chunk(pos_tags)

    # Extract and combine personal names
    names = []
    current_name = []
    for subtree in tree:
        if type(subtree) == nltk.Tree and subtree.label() == 'PERSON':
            current_name.append(" ".join([token for token, pos in subtree.leaves()]))
        elif current_name:
            names.append(" ".join(current_name))
            current_name = []

    if current_name:
        names.append(" ".join(current_name))

    return names

# Example usage
sentence = "I have two friends, Andrew and Emily."
print(extract_personal_names(sentence))

['Andrew', 'Emily']


In [96]:
def make_QnA(filepath):
    
    titles = set(['Mr', 'Mrs', 'Miss', 'Mr.', 'Mrs.', 'Lady', 
              'Sir', 'Lord', 'Don', 'Master', 'Madame', 'Monsieur',
                 'Baron', 'Captain', 'Doctor', 'Dr', 'Dr.', 'Didn', 'Wouldn', 'Isn', 'God'])
    
    with open(filepath, encoding = 'utf-8') as f:
        novel = f.read()
    sentences = list(sent_tokenize(novel))
    propernouns = []
    lengths = []
    ctr = 0
    for s in sentences:
        ctr += 1
        if ctr % 200 == 1:
            print(ctr)
        lengths.append(len(s.split()))
        names = extract_personal_names(s)
        propernouns.append(names)
        
    print('sentences parsed')
    
    questions = []
    answers = []
    
    n = len(sentences)
    ctr = 0
    skip = 0
    
    for idx in range(n-7):
        ctr += 1
        if ctr % 500 == 1:
            print(ctr)
        if skip > 0:
            skip = skip - 1
            continue
            
        wordcount = 0
        namecount = 0
        namelist = []
        for j in range(6):
            wordcount += lengths[idx + j]
            namecount += len(propernouns[idx + j])
            namelist.extend(propernouns[idx + j])
            if wordcount > 40:
                break         
                
        if wordcount > 40 and wordcount < 55 and namecount == 1:
            question = ' '.join(sentences[idx : idx + j + 1])
            answer = namelist[0]
            nameparts = answer.split()
            if len(nameparts) > 1 and nameparts[0] in titles:
                answer = ' '.join(nameparts[1:])
            elif answer in titles:
                continue
                
            question = question.replace(answer, '[MASK]')
            
            if '[MASK]' in question:
                questions.append(question)
                answers.append(answer)
            skip = j
            
    return questions, answers

def select_questions(questions, answers):
    n = len(questions)
    selections = random.sample([x for x in range(n)], 125)
    q = []
    a = []
    for s in selections:
        print(questions[s], ' --- ', answers[s])
        user = input('? ')
        if user != 'n':
            q.append(questions[s])
            a.append(answers[s])
        if len(q) == 100:
            break

    return q, a
        
    

In [108]:
questions, answers = make_QnA('chapterbooks/MurderIsDangerous.txt')

1
201
401
601
801
1001
1201
1401
1601
1801
2001
2201
2401
2601
2801
3001
3201
3401
3601
3801
4001
4201
4401
4601
sentences parsed
1
501
1001
1501
2001
2501
3001
3501
4001
4501


In [109]:
len(questions)

520

In [110]:
q, a = select_questions(questions, answers)

“I wish I did.” I put my arms around her and kissed her. She yielded to me as if she expected this to be our last meeting. When I was able, I opened the door and went out. I waved to her from the pickup and started toward the post office to get [MASK].  ---  Driller


?  


“Go back to sleep.”

When we were outside, [MASK] said, “It could have been one of them.”

“Try to prove it. They’ll stick by each other and you’ll end up by making a lot of enemies and getting no information. I tried it a couple of days ago.  ---  Tawson


?  


Then they decided it might be valuable. They got most of the rest of the land around here. We’re like a sore thumb on their hand.”

A thought occurred to me. “And [MASK]?”

“Naw, I don’t think so. But we can’t be sure.”

I was upset.  ---  Jerome


...

?  


The old man knew everybody who ever had oil fever, so I guessed they had run across each others trails sometime or other. “Well, what do you say, old-timer?” he asked. [MASK] pointed a finger at me. “He’s the party chief.  ---  Driller


?  


“Somebody got there ahead of me and passed the word that I’m poison and shouldn’t be allowed on the property. Both places. Also, whoever it was leased all the water. I wouldn’t doubt that they’ve been paid to pray it doesn’t rain.”

[MASK]’s face showed a faint trace of a smirk.  ---  Jerome


?  


In [106]:
len(q)

100

In [37]:
len(a)

100

In [111]:
with open('QnA/MurderIsDangerous.tsv', mode = 'w', encoding = 'utf-8') as f:
    f.write('question\tanswer\n')
    for question, answer in zip(q, a):
        question = question.replace('\n', ' ').replace('\t', ' ').replace('  ', ' ')
        f.write(question + '\t' + answer + '\n')
        

### Functions for the API

In [47]:
@backoff.on_exception(
    backoff.expo,
    openai.RateLimitError,
    max_time=60,  # Set a maximum wait time in seconds (adjust as needed)
    giveup=lambda e: False  # This prevents giving up on retries
)
def completions_with_backoff(**kwargs):
    global client
    try:
        return client.chat.completions.create(**kwargs)
    except openai.APIError as e:
        print(f"Error: {e}")
        raise  # Re-raise the error to trigger the retry mechanism

In [49]:
def submit_prompt(question, temperature):
    
    system_prompt = "You are identifying characters in fiction. You will receive a \
    passage from a novel, and guess \
    a name that has been replaced by the token [MASK]. Your response will only be the name itself, \
    which can be one or two words. You will get credit if either first or last name is correct."
    
    user1 = '“In fact, I have been meditating on the very great pleasure which a pair of fine eyes in the face of a pretty woman can bestow.” \
    She immediately fixed her eyes on his face, and desired he would tell her what lady had the credit of inspiring such reflections. He replied, with great intrepidity,—\
    “Miss [MASK].”'
    
    assistant1 = 'Elizabeth Bennet'
    
    user2 = "She dropped her thick black veil over her face and glided from the room. \
    “And what do you think of it all, [MASK]?” asked the detective, leaning back in his chair. \
    “It seems to me to be a most dark and sinister business.” \
    “Dark enough and sinister enough, indeed.”"
    
    assistant2 = 'Watson'
    
    prompt = [{"role": "system", "content": system_prompt},
              {"role": "user", "content": user1},
              {"role": "assistant", "content": assistant1},
              {"role": "user", "content": user2},
             {"role": "assistant", "content": assistant2},
             {"role": "user", "content": question}]

    p = list(prompt)
    # print(p)
    try:
        completion = completions_with_backoff(
            model = "gpt-3.5-turbo",
            messages = p,
            max_tokens = 400,
            temperature = temperature
        )
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        
    return completion

In [93]:
def run_test(questions, answers):
    gotright = []
    for question, answer in zip(questions, answers):
        question = question.replace('\n', ' ').replace('\t', ' ').replace('  ', ' ')
        completion = submit_prompt(question, 0.6)
        response = completion.choices[0].message.content
        print(question)
        print(answer)
        print(response, '?')
        
        a_words = answer.split()
        r_words = response.split()
        correct = False
        for r in r_words:
            if r in a_words:
                correct = True
        print(correct)
        print()
        gotright.append(correct)
    
    print(sum(gotright) / len(gotright))
    
    return gotright
        
                

In [112]:
gotright = run_test(q, a)

“I wish I did.” I put my arms around her and kissed her. She yielded to me as if she expected this to be our last meeting. When I was able, I opened the door and went out. I waved to her from the pickup and started toward the post office to get [MASK].
Driller
the mail ?
False

“Go back to sleep.” When we were outside, [MASK] said, “It could have been one of them.” “Try to prove it. They’ll stick by each other and you’ll end up by making a lot of enemies and getting no information. I tried it a couple of days ago.
Tawson
Marlowe ?
False

Then they decided it might be valuable. They got most of the rest of the land around here. We’re like a sore thumb on their hand.” A thought occurred to me. “And [MASK]?” “Naw, I don’t think so. But we can’t be sure.” I was upset.
Jerome
John ?
False

Did you lock these doors before you left?” He looked in. Then he looked at me. His expression hadn’t changed. “I don’t remember. Why?” [MASK] and I didn’t go into the details. We simply told him how we ha